# Linear IVP adjoint example

This example calculates the optimal initial perturbation $\mathbf{u}(0)=\mathbf{u}_0$ which maximises $\mathbf{u}_T=\mathbf{u}(T)$. In other words we maximise $\mathcal{J}=\langle \mathbf{u}_T,\mathbf{W} \mathbf{u}_T\rangle/\langle \mathbf{u}_0,\mathbf{W} \mathbf{u}_0\rangle$, where $\mathbf{W}$ is a weight matrix, subject to $\mathbf{u}=(u,v,w)^T$ solving
\begin{eqnarray}
\frac{\partial u}{\partial t} + i\alpha u u_0 + u \frac{du_0}{dy} &=& i\alpha p + \frac{1}{\textit{Re}}\left(-\alpha^2u-\beta^2u + \frac{d^2u}{dy^2}\right),\\
\frac{\partial v}{\partial t} + i\alpha v u_0 &=& \frac{dp}{dy} + \frac{1}{\textit{Re}}\left(-\alpha^2v-\beta^2v + \frac{d^2v}{dy^2}\right),\\
\frac{\partial w}{\partial t} + i\alpha w u_0 &=& i\beta p + \frac{1}{\textit{Re}}\left(-\alpha^2w-\beta^2w + \frac{d^2w}{dy^2}\right),\\
i\alpha u + \frac{dv}{dy}+i\beta w &=& 0.
\end{eqnarray}
$\mathbf{W}$ is chosen so that $\langle \mathbf{u},\mathbf{W} \mathbf{u}\rangle\approx\int \bar{\mathbf{u}}\cdot  \mathbf{u}\;\textrm{d}y$.

The notebook is setup to calculate the stable line in figure 4.3 a) of 'Stability and transition in shear flows', Schmid and Henningson. 

In [1]:
import matplotlib.pyplot as plt
import dedalus.public as d3
import logging
import numpy as np

from dedalus.tools import jacobi
from dedalus.core import operators
from scipy import optimize

logger = logging.getLogger(__name__)

Set up the IVP that will find $\mathbf{u}_T$ from $\mathbf{u}_0$

In [2]:
# Parameters
Ny = 128
dtype = np.complex128

# Bases
coords = d3.CartesianCoordinates('y')
dist = d3.Distributor(coords, dtype=dtype)
ybasis = d3.ChebyshevT(coords['y'], size=Ny,dealias=1, bounds=(0, 2))

y, = dist.local_grids(ybasis)

dy = lambda A: d3.Differentiate(A, coords['y'])

# Fields
u = dist.Field(name='u', bases=(ybasis))
v = dist.Field(name='v', bases=(ybasis))
w = dist.Field(name='w', bases=(ybasis))

u0 = dist.Field(name='u0', bases=(ybasis))
v0 = dist.Field(name='v0', bases=(ybasis))
w0 = dist.Field(name='w0', bases=(ybasis))

gradu = dist.Field(name='gradu', bases=(ybasis))
gradv = dist.Field(name='gradv', bases=(ybasis))
gradw = dist.Field(name='gradw', bases=(ybasis))

U = dist.Field(name='U',bases=(ybasis))
U['g'] = y*(2-y)
Uy = dy(U)
alpha = 1
beta = 0
p = dist.Field(name='p', bases=(ybasis))
tau_u_1 = dist.Field(name='tau_u_1')
tau_u_2 = dist.Field(name='tau_u_2')

tau_v_1 = dist.Field(name='tau_v_1')
tau_v_2 = dist.Field(name='tau_v_2')

tau_w_1 = dist.Field(name='tau_w_1')
tau_w_2 = dist.Field(name='tau_w_2')

tau_p = dist.Field(name='tau_p')

ybasis2 = ybasis.derivative_basis(2)

# # Substitutions
lift_basis = ybasis.derivative_basis(2)
lift = lambda A, n: d3.Lift(A, lift_basis, n)
Re = 5000

# # Problem
problem = d3.IVP([u,v,w,p, tau_u_1, tau_u_2,tau_v_1, tau_v_2,tau_w_1, tau_w_2], namespace=locals())
problem.add_equation("dt(u) + 1j*alpha*u*U + v*Uy - 1/Re*(dy(dy(u))-alpha**2*u-beta**2*u) + lift(tau_u_1,-1) + lift(tau_u_2,-2) + 1j*alpha*p = 0")
problem.add_equation("dt(v) + 1j*alpha*v*U - 1/Re*(dy(dy(v))-alpha**2*v-beta**2*v) + lift(tau_v_1,-1) + lift(tau_v_2,-2) + dy(p) = 0")
problem.add_equation("dt(w) + 1j*alpha*w*U - 1/Re*(dy(dy(w))-alpha**2*w-beta**2*w) + lift(tau_w_1,-1) + lift(tau_w_2,-2) + 1j*beta*p = 0")
problem.add_equation("1j*alpha*u + dy(v) + 1j*beta*w = 0")

problem.add_equation("u(y=0) = 0")
problem.add_equation("u(y=2) = 0")

problem.add_equation("v(y=0) = 0")
problem.add_equation("v(y=2) = 0")

problem.add_equation("w(y=0) = 0")
problem.add_equation("w(y=2) = 0")

## Test
First we test forward and adjoint solves using the fact that $\langle \mathbf{u}_T^\dagger,\mathbf{u}_T\rangle=\langle \mathbf{u}_0^\dagger,\mathbf{u}_0\rangle$.

In [3]:
a0 = np.random.rand(Ny)+1j*np.random.rand(Ny)
a1 = (np.random.rand(Ny)+1j*np.random.rand(Ny))
a2 = (np.random.rand(Ny)+1j*np.random.rand(Ny))

b0 = np.random.rand(Ny)+1j*np.random.rand(Ny)
b1 = (np.random.rand(Ny)+1j*np.random.rand(Ny))
b2 = (np.random.rand(Ny)+1j*np.random.rand(Ny))

In [4]:
u['c'] = a0
v['c'] = a1
w['c'] = a2
solver = problem.build_solver(d3.SBDF4)

for (i,state) in enumerate(solver.state):
    if(i>2):
        print(state)
        state['g'] = 0

solver.stop_iteration = 100
timestep = 0.1
try:
    logger.info('Starting main loop')
    while solver.proceed:
        solver.step(timestep*np.abs(np.random.rand()))
#         solver.step(timestep)
except:
    logger.error('Exception raised, triggering end of main loop.')
    raise
finally:
    solver.log_stats()
Ga0 = u['c']
Ga1 = v['c']
Ga2 = w['c']

2023-05-10 17:23:26,712 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 8.5e+00/s
p
tau_u_1
tau_u_2
tau_v_1
tau_v_2
tau_w_1
tau_w_2
2023-05-10 17:23:26,715 __main__ 0/1 INFO :: Starting main loop
2023-05-10 17:23:26,843 solvers 0/1 INFO :: Stop iteration reached.
2023-05-10 17:23:26,843 solvers 0/1 INFO :: Final iteration: 100
2023-05-10 17:23:26,844 solvers 0/1 INFO :: Final sim time: 5.490908829651666
2023-05-10 17:23:26,845 solvers 0/1 INFO :: Setup time (init - iter 0): 0.1407 sec
2023-05-10 17:23:26,845 solvers 0/1 INFO :: Warmup time (iter 0-10): 0.02038 sec
2023-05-10 17:23:26,846 solvers 0/1 INFO :: Run time (iter 10-end): 0.1071 sec
2023-05-10 17:23:26,847 solvers 0/1 INFO :: CPU time (iter 10-end): 2.975e-05 cpu-hr
2023-05-10 17:23:26,848 solvers 0/1 INFO :: Speed: 4.353e+05 mode-stages/cpu-sec


In [5]:
solver.state_adj[0]['c'] = b0
solver.state_adj[1]['c'] = b1
solver.state_adj[2]['c'] = b2

for (i,state) in enumerate(solver.state_adj):
    if(i>2):
        print(state)
        state['g'] = 0

try:
    logger.info('Starting adjoint loop')
    while solver.iteration>0:
        solver.step_adjoint()

except:
    logger.error('Exception raised, triggering end of main loop.')
    raise
finally:
    solver.log_stats()
    
GTb0 = solver.state_adj[0]['c']
GTb1 = solver.state_adj[1]['c']
GTb2 = solver.state_adj[2]['c']

p_adj
tau_u_1_adj
tau_u_2_adj
tau_v_1_adj
tau_v_2_adj
tau_w_1_adj
tau_w_2_adj
2023-05-10 17:23:26,856 __main__ 0/1 INFO :: Starting adjoint loop
2023-05-10 17:23:26,858 solvers 0/1 INFO :: Warning: Only works if F==0 and linear direct equation.
2023-05-10 17:23:27,246 solvers 0/1 INFO :: Final iteration: 0
2023-05-10 17:23:27,247 solvers 0/1 INFO :: Final sim time: 1.8735013540549517e-16
2023-05-10 17:23:27,247 solvers 0/1 INFO :: Setup time (init - iter 0): 0.1407 sec
2023-05-10 17:23:27,248 solvers 0/1 INFO :: Timings unavailable because warmup did not complete.


In [6]:
term1 = np.vdot(b0,Ga0) +  np.vdot(b1,Ga1) +  np.vdot(b2,Ga2)
term2 = np.vdot(GTb0,a0) + np.vdot(GTb1,a1) + np.vdot(GTb2,a2)

In [7]:
print(term1,term2)
print('error =',np.abs(term1-term2)/np.abs(term1))

(0.3462145580684196+0.8173668301405659j) (0.34621455806833684+0.8173668301403697j)
error = 2.3986640070777424e-13
